In [1]:
import os
import pandas as pd

# Tous les fichiers dans le dossier téléchargé
directory = "C:/Users/camil/IMAG/DeepDRA/GDC"

# Liste tous les fichiers .maf.gz dans tous les sous-dossiers
maf_files = [
    os.path.join(root, file)
    for root, dirs, files in os.walk(directory)
    for file in files
    if file.endswith(".maf.gz")
]

print(f"Nombre de fichiers MAF trouvés : {len(maf_files)}")


Nombre de fichiers MAF trouvés : 675


In [2]:
# Charger tous les MAF dans une seule liste de DataFrames
dfs = []
failed_files = []

for f in maf_files:
    try:
        df = pd.read_csv(f, sep="\t", comment="#", low_memory=False)
        if not df.empty:
            dfs.append(df)
    except Exception as e:
        print(f"Erreur avec le fichier : {f}\n→ {e}")
        failed_files.append(f)

df_all = pd.concat(dfs, ignore_index=True)

print(f" {len(dfs)} fichiers fusionnés.")
print(f" {len(failed_files)} fichiers ignorés.")
print(f" Shape finale du DataFrame : {df_all.shape}")

 671 fichiers fusionnés.
 0 fichiers ignorés.
 Shape finale du DataFrame : (137368, 140)


In [3]:
# Charger la table de correspondance
id_map = pd.read_csv("tcga_ids_mapping.tsv", sep="\t")

# Fusionner avec df_all sur la colonne 'case_id'
df_all = df_all.merge(id_map, on="case_id", how="left")

In [4]:
# Supprimer les colonnes
df_all_filtered = df_all.drop(columns=["Center", "NCBI_Build", 'dbSNP_RS', 'dbSNP_Val_Status', 'Match_Norm_Seq_Allele1', 'Match_Norm_Seq_Allele2', 'Tumor_Validation_Allele1', 'Tumor_Validation_Allele2', 'Match_Norm_Validation_Allele1', 'Match_Norm_Validation_Allele2', 'Verification_Status', 'Validation_Status', 'Sequencing_Phase', 'Sequence_Source', 'Validation_Method', 'Score', 'BAM_File', 'Sequencer', 'n_ref_count', 'n_alt_count', 'DISTANCE', 'SYMBOL_SOURCE', 'TREMBL', 'FLAGS', 'INTRON', '1000G_AF', '1000G_AFR_AF', '1000G_AMR_AF', '1000G_EAS_AF', '1000G_EUR_AF', '1000G_SAS_AF', 'ESP_AA_AF', 'ESP_EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'gnomAD_non_cancer_AF', 'gnomAD_non_cancer_AFR_AF', 'gnomAD_non_cancer_AMI_AF', 'gnomAD_non_cancer_AMR_AF', 'gnomAD_non_cancer_ASJ_AF', 'gnomAD_non_cancer_EAS_AF', 'gnomAD_non_cancer_FIN_AF', 'gnomAD_non_cancer_MID_AF', 'gnomAD_non_cancer_NFE_AF', 'gnomAD_non_cancer_OTH_AF', 'gnomAD_non_cancer_SAS_AF', 'gnomAD_non_cancer_MAX_AF_adj', 'gnomAD_non_cancer_MAX_AF_POPS_adj', 'PUBMED', 'TRANSCRIPTION_FACTORS', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'miRNA', 'HGVS_OFFSET', 'case_id', 'GDC_FILTER', 'RNA_Support', 'RNA_depth', 'RNA_ref_count', 'RNA_alt_count'])

In [5]:
df_all_filtered.shape

(137368, 70)

In [ ]:
# Charger la liste des gènes d'intérêt à partir de TCGA_mutation_gene_list.csv
gene_list_df = pd.read_csv("TCGA_mutation_gene_list.csv")

# Lire les gènes depuis une ligne unique sans en-tête
gene_list_df = pd.read_csv("TCGA_mutation_gene_list.csv", header=None)

# Extraire la première ligne sous forme de liste de gènes
gene_list = set(gene_list_df.iloc[0].values)

# Extraire les gènes uniques du DataFrame MAF filtré
df_genes = set(df_all_filtered['Hugo_Symbol'].unique())

# Intersection : gènes en commun
matching_genes = df_genes.intersection(gene_list)

In [8]:
# Affichage des résultats
print(f" Nombre de gènes uniques dans le MAF : {len(df_genes)}")
print(f" Nombre de gènes dans la liste de référence : {len(gene_list)}")
print(f" Nombre de gènes en commun : {len(matching_genes)}")

 Nombre de gènes uniques dans le MAF : 17798
 Nombre de gènes dans la liste de référence : 19100
 Nombre de gènes en commun : 15458


In [9]:
# Filtrer le DataFrame pour ne garder que les lignes dont le gène est dans la liste des gènes communs
df_all_filtered = df_all_filtered[df_all_filtered['Hugo_Symbol'].isin(matching_genes)]

# Afficher le résultat
print(f" Nombre de mutations conservées après filtrage : {df_all_filtered.shape[0]}")
print(f" Nombre de gènes uniques dans ce sous-ensemble : {df_all_filtered['Hugo_Symbol'].nunique()}")


 Nombre de mutations conservées après filtrage : 124534
 Nombre de gènes uniques dans ce sous-ensemble : 15458


In [10]:
# Supprimer les mutations silencieuses
df_all_filtered = df_all_filtered[df_all_filtered["Variant_Classification"] != "Silent"]

print(f" Nombre de mutations après exclusion des 'Silent' : {df_all_filtered.shape[0]}")

 Nombre de mutations après exclusion des 'Silent' : 95706


In [11]:
df_all_filtered.shape


(95706, 70)

In [12]:
# Créer la matrice binaire (patients x gènes)
# Chaque ligne = un patient (entity_submitter_id), chaque colonne = un gène
binary_matrix = (
    df_all_filtered.groupby(["entity_submitter_id", "Hugo_Symbol"])
    .size()  # compte le nombre de mutations par couple
    .unstack(fill_value=0)  # transforme en table avec 0 pour les absents
)

# Binariser : 1 si au moins une mutation dans le gène, sinon 0
binary_matrix = (binary_matrix > 0).astype(int)

# Renommer les colonnes avec suffixe '_cell_mut'
binary_matrix.columns = [f"{gene}_cell_mut" for gene in binary_matrix.columns]

print(f" Matrice binaire créée. Shape: {binary_matrix.shape}")

 Matrice binaire créée. Shape: (664, 15009)


In [13]:
# Sauvegarder en TSV
output_path = "C:/Users/camil/IMAG/DeepDRA/TCGA_data/TCGA_data/cell_mut_raw.tsv"
binary_matrix.to_csv(output_path, sep="\t")
